This notebook demonstrates a prototype of a TRX format that leverages the parquet format.

In [1]:
from trx_polars import trxparquet

The primary class is `TrxParquet`. It contains two key attributes, `header` and `data`. The attribute `header` is an attempt at storing a minimal amount of information necessary for processing and conversion to other formats. The `data` attribute is where most of the information is contained.

One quick way to get started is via `init_example_trxparquet`, which initializes trxparquet files with given characteristics and random data. For example, we can start with a representation of two streamlines, each having 3 points.

In [2]:
trxparquet.init_example_trxparquet(2, 3)

TrxParquet(header=TrxHeader(DIMENSIONS=array([20, 20, 20], dtype=uint16), VOXEL_TO_RASMM=array([[20.,  0.,  0.,  0.],
       [ 0., 20.,  0.,  0.],
       [ 0.,  0., 20.,  0.],
       [ 0.,  0.,  0.,  1.]], dtype=float32)), data=shape: (6, 4)
┌──────────────────────┬──────────────────────┬──────────────────────┬──────────────────────┐
│ protected_streamline ┆ protected_position_0 ┆ protected_position_1 ┆ protected_position_2 │
│ ---                  ┆ ---                  ┆ ---                  ┆ ---                  │
│ i64                  ┆ f64                  ┆ f64                  ┆ f64                  │
╞══════════════════════╪══════════════════════╪══════════════════════╪══════════════════════╡
│ 0                    ┆ 0.448707             ┆ 0.526922             ┆ 0.146535             │
│ 0                    ┆ 0.925829             ┆ 0.184845             ┆ 0.762281             │
│ 0                    ┆ 0.176739             ┆ 0.202632             ┆ 0.701005             │
│ 1   

In the parquet file, the `header` is stored via frame-level metadata (e.g., of the kind readable by https://arrow.apache.org/docs/python/generated/pyarrow.parquet.read_metadata.html). The `header` will not be discussed further in this notebook. 

The `data` always contains at least four columns, each of which have the prefix "protected_". These columns represent 
- An index for streamline
- 3 columns representing the coordinates of each point/vertex within each streamline. 

That is, rows in the data correspond to points or vertices.

Data that is associated with each streamline will be stored in a column that has a prefix "dps_". The function `init_example_trxparquet` can be used to create a this kind of column. The label with include a random string.

In [3]:
trxparquet.init_example_trxparquet(2, 3, 1)

TrxParquet(header=TrxHeader(DIMENSIONS=array([20, 20, 20], dtype=uint16), VOXEL_TO_RASMM=array([[20.,  0.,  0.,  0.],
       [ 0., 20.,  0.,  0.],
       [ 0.,  0., 20.,  0.],
       [ 0.,  0.,  0.,  1.]], dtype=float32)), data=shape: (6, 5)
┌─────────────────────┬─────────────────────┬────────────────────┬────────────────────┬────────────┐
│ protected_streamlin ┆ protected_position_ ┆ protected_position ┆ protected_position ┆ dps_ahtcbm │
│ e                   ┆ 0                   ┆ _1                 ┆ _2                 ┆ ---        │
│ ---                 ┆ ---                 ┆ ---                ┆ ---                ┆ f64        │
│ i64                 ┆ f64                 ┆ f64                ┆ f64                ┆            │
╞═════════════════════╪═════════════════════╪════════════════════╪════════════════════╪════════════╡
│ 0                   ┆ 0.886641            ┆ 0.521663           ┆ 0.247088           ┆ 0.43597    │
│ 0                   ┆ 0.125199            ┆ 0.879

Analogously, data that is associated with individual points will have the prefix "dpv_".

In [4]:
trxparquet.init_example_trxparquet(2, 3, 1, 1)

TrxParquet(header=TrxHeader(DIMENSIONS=array([20, 20, 20], dtype=uint16), VOXEL_TO_RASMM=array([[20.,  0.,  0.,  0.],
       [ 0., 20.,  0.,  0.],
       [ 0.,  0., 20.,  0.],
       [ 0.,  0.,  0.,  1.]], dtype=float32)), data=shape: (6, 6)
┌──────────────────┬─────────────────┬─────────────────┬─────────────────┬────────────┬────────────┐
│ protected_stream ┆ protected_posit ┆ protected_posit ┆ protected_posit ┆ dps_alczhe ┆ dpv_uqynzi │
│ line             ┆ ion_0           ┆ ion_1           ┆ ion_2           ┆ ---        ┆ ---        │
│ ---              ┆ ---             ┆ ---             ┆ ---             ┆ f64        ┆ f64        │
│ i64              ┆ f64             ┆ f64             ┆ f64             ┆            ┆            │
╞══════════════════╪═════════════════╪═════════════════╪═════════════════╪════════════╪════════════╡
│ 0                ┆ 0.734908        ┆ 0.304299        ┆ 0.640699        ┆ 0.246523   ┆ 0.041026   │
│ 0                ┆ 0.080363        ┆ 0.441091    

To see this value in a more familiar format, they can be converted into `StatefulTractogram`s using the `to_stf()` method.

In [5]:
stf = trxparquet.init_example_trxparquet(2, 3, 1, 1).to_stf()
stf.streamlines

ArraySequence([array([[0.94635898, 0.85049711, 0.51803166],
       [0.25558301, 0.33324491, 0.68452373],
       [0.15770856, 0.97145055, 0.40997666]]), array([[0.5238964 , 0.21003059, 0.55388576],
       [0.16877643, 0.7876155 , 0.92684308],
       [0.3443854 , 0.79540141, 0.8014279 ]])])

The assumption is that, by relying on the parquet format, we get get to leverage all of the work that has gone into making this an efficient medium for analysis. For example, let's create a file that has 100000 streamlines, each with 100 points, checking the size of the file and how long it takes to load.

In [6]:
import tempfile
import time
from pathlib import Path


def human_size(bytes, units=[" bytes", "KB", "MB", "GB", "TB", "PB", "EB"]):
    """Returns a human readable string representation of bytes"""
    return str(bytes) + units[0] if bytes < 1024 else human_size(bytes >> 10, units[1:])


with tempfile.NamedTemporaryFile(suffix=".parquet") as _f:
    f = Path(_f.name)
    trx = trxparquet.init_example_trxparquet(100000, 100)
    trx.to_file(f)
    size = human_size(f.stat().st_size)
    start = time.time()
    trx2 = trx.from_file(f, loadtype="memory_map")
    end = time.time()

print(f"size: {size}")
print(f"reading time: {end - start}")

trx2
del trx, trx2

size: 230MB
reading time: 0.13715624809265137


There are different ways of loading parquet files, each optimized for different purposes. The previous cell loaded files as a memory map. If only some streamlines need to be processed (or only some columns), then Lazy loading has many advantages. See: https://pola-rs.github.io/polars/user-guide/concepts/lazy-vs-eager/ . The reading time for lazy loading is minimal, but we can still extract useful information from the result.

In [7]:
with tempfile.NamedTemporaryFile(suffix=".parquet") as _f:
    f = Path(_f.name)
    trx = trxparquet.init_example_trxparquet(100000, 100)
    trx.to_file(f)
    start = time.time()
    n_streamlines_in_file = trx.from_file(f, loadtype="lazy").n_streamlines
    end = time.time()

print(f"reading time: {end - start}")
print(f"{n_streamlines_in_file=}")
del trx

reading time: 0.04859614372253418
n_streamlines_in_file=100000


Even when files are loaded into memory, the process is remains speedy.

In [8]:
with tempfile.NamedTemporaryFile(suffix=".parquet") as _f:
    f = Path(_f.name)
    trx = trxparquet.init_example_trxparquet(100000, 100)
    trx.to_file(f)
    start = time.time()
    trx2 = trx.from_file(f, loadtype="memory")
    end = time.time()

object_size = trx2.data.estimated_size("mb")
print(f"{object_size=} MB")
print(f"reading time: {end - start}")
del trx, trx2

object_size=305.17578125 MB
reading time: 0.11124205589294434


For a few additional examples, please see the tests.

Note that, at the time of writing, no group-level information has been incorporated into the prototype.